# Sourcery Quickstart Notebook

Schema-first extraction from inline text with grounded spans.

In [ ]:
%pip install -q sourcery pydantic

In [ ]:
from pydantic import BaseModel
import sourcery
from sourcery.contracts import (
    EntitySchemaSet,
    EntitySpec,
    ExtractRequest,
    ExtractionExample,
    ExtractionTask,
    ExampleExtraction,
    RuntimeConfig,
)


class PersonAttrs(BaseModel):
    role: str | None = None


request = ExtractRequest(
    documents="Alice Johnson is the CEO of Acme Robotics.",
    task=ExtractionTask(
        instructions="Extract person entities with role.",
        schema=EntitySchemaSet(entities=[EntitySpec(name="person", attributes_model=PersonAttrs)]),
        examples=[
            ExtractionExample(
                text="Bob is the CTO.",
                extractions=[
                    ExampleExtraction(entity="person", text="Bob", attributes={"role": "CTO"})
                ],
            )
        ],
    ),
    runtime=RuntimeConfig(model="deepseek/deepseek-chat"),
)

result = sourcery.extract(request)
result.metrics.model_dump(mode="json")

In [ ]:
[
    (e.entity, e.text, e.char_start, e.char_end, e.alignment_status)
    for e in result.documents[0].extractions
]

In [ ]:
from pathlib import Path
from sourcery.io import save_extract_result_jsonl, write_document_html, write_reviewer_html

output_dir = Path("examples/output/notebook_quickstart")
output_dir.mkdir(parents=True, exist_ok=True)

jsonl_path = output_dir / "result.jsonl"
viewer_path = output_dir / "result.viewer.html"
reviewer_path = output_dir / "result.reviewer.html"

save_extract_result_jsonl(result, jsonl_path)
write_document_html(result.documents[0], viewer_path)
write_reviewer_html(result.documents[0], reviewer_path)

str(jsonl_path), str(viewer_path), str(reviewer_path)